# Bayesian Optimization Case Study

In this case study, we will explore how to use Bayesian Optimization to tune the hyperparameters of a machine learning model. Bayesian Optimization is a powerful technique for optimizing expensive black-box functions, such as the validation performance of a complex model. It works by constructing a probabilistic model of the objective function and using it to guide the search for the optimal hyperparameter values.


We will be using the LightGBM library to build a binary classification model on a real-world dataset. Our goal is to find the best hyperparameter settings that maximize the model's AUC score using Bayesian Optimization.

## Setup

First, let's import the necessary libraries and load the data:

In [1]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import lightgbm as lgb
from bayes_opt import BayesianOptimization
from sklearn.model_selection import train_test_split

In [2]:
train_df = pd.read_csv('flight_delays_train.csv')
test_df = pd.read_csv('flight_delays_test.csv')

train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [3]:
train_df.describe()

,DepTime,Distance
count,100000.000000,100000.00000
mean,1341.523880,729.39716
std,476.378445,574.61686
min,1.000000,30.00000
25%,931.000000,317.00000
50%,1330.000000,575.00000
75%,1733.000000,957.00000
max,2534.000000,4962.00000


The response variable is 'dep_delayed_15min' which is a categorical column, so we need to map the Y for yes and N for no values to 1 and 0.

In [4]:
y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values

## Feature Engineering

The feature engineering process involves creating new features and transforming existing ones to better capture the underlying patterns in the data, aiming to improve model performance. The steps include:

- **Combining Columns**: New identifiers are created by concatenating `Origin` and `Dest` into `flight`, and similarly for `flightUC`, `DestUC`, and `OriginUC` with `UniqueCarrier`.
- **Extracting Date Parts**: Numerical parts are extracted from `Month`, `DayofMonth`, and `DayOfWeek`, and categorized into beginning, middle, or end of the month, and into seasons.
- **Time Transformations**: `DepTime` is converted into `hour`, and further into parts of the day (`morning`, `day`, `evening`, `night`), and harmonic features (`deptime_cos`, `deptime_sin`) to capture cyclical nature.
- **Categorical to Numerical**: Categorical variables like `UniqueCarrier`, `Origin`, and `Dest` are encoded numerically.
- **Aggregations**: Counts per month and overall for airports (`Dest`, `Origin`) and carriers are calculated to capture frequency information.
- **Dropping Unused Columns**: Columns like `DepTime` are dropped after extracting relevant features.

The `label_enc` function applies label encoding to a given DataFrame column, converting categorical variables into numerical representations.

The `make_harmonic_features_sin` and `make_harmonic_features_cos` functions create sine and cosine features based on the `DepTime` column, which can help capture periodic patterns in the data.

The `feature_eng` function performs the feature engineering steps described above, creating new columns and transforming existing ones to enrich the dataset for the predictive model.


In [5]:
def label_enc(df_column):
    df_column = LabelEncoder().fit_transform(df_column)
    return df_column

def make_harmonic_features_sin(value, period=2400):
    value *= 2 * np.pi / period 
    return np.sin(value)

def make_harmonic_features_cos(value, period=2400):
    value *= 2 * np.pi / period 
    return np.cos(value)

def feature_eng(df):
    df['flight'] = df['Origin']+df['Dest']
    df['Month'] = df.Month.map(lambda x: x.split('-')[-1]).astype('int32')
    df['DayofMonth'] = df.DayofMonth.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['begin_of_month'] = (df['DayofMonth'] < 10).astype('uint8')
    df['midddle_of_month'] = ((df['DayofMonth'] >= 10)&(df['DayofMonth'] < 20)).astype('uint8')
    df['end_of_month'] = (df['DayofMonth'] >= 20).astype('uint8')
    df['DayOfWeek'] = df.DayOfWeek.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['hour'] = df.DepTime.map(lambda x: x/100).astype('int32')
    df['morning'] = df['hour'].map(lambda x: 1 if (x <= 11)& (x >= 7) else 0).astype('uint8')
    df['day'] = df['hour'].map(lambda x: 1 if (x >= 12) & (x <= 18) else 0).astype('uint8')
    df['evening'] = df['hour'].map(lambda x: 1 if (x >= 19) & (x <= 23) else 0).astype('uint8')
    df['night'] = df['hour'].map(lambda x: 1 if (x >= 0) & (x <= 6) else 0).astype('int32')
    df['winter'] = df['Month'].map(lambda x: x in [12, 1, 2]).astype('int32')
    df['spring'] = df['Month'].map(lambda x: x in [3, 4, 5]).astype('int32')
    df['summer'] = df['Month'].map(lambda x: x in [6, 7, 8]).astype('int32')
    df['autumn'] = df['Month'].map(lambda x: x in [9, 10, 11]).astype('int32')
    df['holiday'] = (df['DayOfWeek'] >= 5).astype(int) 
    df['weekday'] = (df['DayOfWeek'] < 5).astype(int)
    df['airport_dest_per_month'] = df.groupby(['Dest', 'Month'])['Dest'].transform('count')
    df['airport_origin_per_month'] = df.groupby(['Origin', 'Month'])['Origin'].transform('count')
    df['airport_dest_count'] = df.groupby(['Dest'])['Dest'].transform('count')
    df['airport_origin_count'] = df.groupby(['Origin'])['Origin'].transform('count')
    df['carrier_count'] = df.groupby(['UniqueCarrier'])['Dest'].transform('count')
    df['carrier_count_per month'] = df.groupby(['UniqueCarrier', 'Month'])['Dest'].transform('count')
    df['deptime_cos'] = df['DepTime'].map(make_harmonic_features_cos)
    df['deptime_sin'] = df['DepTime'].map(make_harmonic_features_sin)
    df['flightUC'] = df['flight']+df['UniqueCarrier']
    df['DestUC'] = df['Dest']+df['UniqueCarrier']
    df['OriginUC'] = df['Origin']+df['UniqueCarrier']
    return df.drop('DepTime', axis=1)

In [6]:
# Concatenate the training and test data
full_df = pd.concat([train_df.drop('dep_delayed_15min', axis=1), test_df])
full_df = feature_eng(full_df)

# Apply the feature engineering functions to the full dataframe
for column in ['UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']:
    full_df[column] = label_enc(full_df[column])

# Split the new full dataframe into X_train and X_test. 
X_train = full_df[:train_df.shape[0]]
X_test = full_df[train_df.shape[0]:]

# Create a list of the categorical features.
categorical_features = ['Month',  'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']

## LightGBM
The `lgb` library refers to LightGBM, a gradient boosting framework that uses tree-based learning algorithms. It is designed for speed and efficiency, suitable for large datasets.

The `lgb_eval` function is designed to evaluate a LightGBM model with specified hyperparameters using cross-validation. It returns the mean AUC (Area Under the ROC Curve) score from the last iteration of the cross-validation process, which serves as a performance metric for binary classification tasks.

### Parameters:
- `num_leaves`: The number of leaves in each tree. More leaves can increase model complexity.
- `max_depth`: The maximum depth of each tree. A deeper tree can express more complex relationships.
- `lambda_l2`: L2 regularization term on weights, used to avoid overfitting.
- `lambda_l1`: L1 regularization term on weights, also used to prevent overfitting.
- `min_child_samples`: The minimum number of data points needed in a leaf.
- `min_data_in_leaf`: The minimum number of data points allowed in a leaf. This can be used to deal with over-fitting.
- `learning_rate`: The step size at each iteration while moving toward a minimum of a loss function.
- `subsample_freq`: The frequency for bagging.
- `bagging_seed`: Random seed for bagging.
- `verbosity`: The level of verbosity of the process (e.g., -1 for no output).

### Usage:
This function is used for hyperparameter tuning in the context of a Bayesian optimization process. By evaluating the model's performance with different sets of hyperparameters, one can identify the optimal configuration that maximizes the AUC score. This is particularly useful in machine learning tasks where model performance is critical, such as predicting flight delays in the provided context.

In [7]:
def lgb_eval(num_leaves, max_depth, lambda_l2, lambda_l1, min_child_samples, min_data_in_leaf):
    """Evaluate LightGBM model with given hyperparameters using cross-validation."""
    params = {
        "objective": "binary",
        "metric": "auc",
        'is_unbalance': True,
        "num_leaves": int(num_leaves),
        "max_depth": int(max_depth),
        "lambda_l2": lambda_l2,
        "lambda_l1": lambda_l1,
        "num_threads": 20,
        "min_child_samples": int(min_child_samples),
        'min_data_in_leaf': int(min_data_in_leaf),
        "learning_rate": 0.03,
        "subsample_freq": 5,
        "bagging_seed": 42,
        "verbosity": -1
    }

    lgtrain = lgb.Dataset(X_train, y_train, categorical_feature=categorical_features)

    early_stopping_callback = lgb.early_stopping(50, verbose=False)  # Stops if no improvement after 50 rounds

    cv_result = lgb.cv(params,
                       lgtrain,
                       num_boost_round=1000,
                       nfold=3,
                       stratified=True,
                       seed=42,
                       callbacks=[early_stopping_callback])
    
    return cv_result['valid auc-mean'][-1]

Next we'll apply the Bayesian optimizer to the function we created in the previous step to identify the best hyperparameters. We will run 10 iterations and set init_points = 2.


In [8]:
# Setup for Bayesian Optimization
lgbBO = BayesianOptimization(lgb_eval, {
    'num_leaves': (25, 4000),
    'max_depth': (5, 63),
    'lambda_l2': (0.0, 0.05),
    'lambda_l1': (0.0, 0.05),
    'min_child_samples': (50, 10000),
    'min_data_in_leaf': (100, 2000)
})

lgbBO.maximize(n_iter=10, init_points=2)

|   iter    |  target   | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_da... | num_le... |
-------------------------------------------------------------------------------------------------
| 1         | 0.721     | 1.584e-05 | 0.02098   | 12.97     | 2.514e+03 | 477.1     | 3.915e+03 |
| 2         | 0.7448    | 0.02695   | 0.03285   | 31.97     | 6.261e+03 | 1.358e+03 | 843.9     |
| 3         | 0.7443    | 0.01316   | 0.03652   | 58.27     | 605.8     | 1.842e+03 | 388.6     |
| 4         | 0.7272    | 0.01684   | 0.03403   | 46.01     | 4.214e+03 | 807.1     | 2.991e+03 |
| 5         | 0.7444    | 0.008681  | 0.04803   | 39.03     | 9.12e+03  | 1.22e+03  | 590.2     |
| 6         | 0.7444    | 0.04788   | 0.009086  | 15.16     | 9.807e+03 | 1.978e+03 | 3.994e+03 |
| 7         | 0.7406    | 0.002467  | 0.03286   | 55.81     | 6.036e+03 | 1.151e+03 | 997.6     |
| 8         | 0.7445    | 0.002903  | 0.02298   | 13.28     | 4.079e+03 | 1.438e+03 | 1.743e+03 |
| 9         | 0.7445

In [ ]:
# Print the best results
print(lgbBO.max)

# Review the process at each step using the '.res[0]' function
print(lgbBO.res[0])

